In [1]:
%pip install 'psycopg[binary]'


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
from pathlib import Path
from decouple import config


In [3]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [4]:
from sqlmodel import Session, select, desc
from api.db.session import engine
from api.events.models import EventModel

In [5]:
with Session(engine) as session:
    query = select(EventModel).order_by(desc(EventModel.updated_at)).limit(10)
    compile_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compile_query)
    print("---")
    print(str(query))
    # results = session.exec(query).all()
    # print(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT 10
---
SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT :param_1


In [ ]:
from timescaledb.hyperfunctions import time_bucket, time_bucket_gapfill
from pprint import pprint
from datetime import datetime, timedelta, timezone
from sqlalchemy import func
with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    pages = ['/about', '/contact', '/home', '/products', '/services']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(minutes=1)
    query = select(bucket, EventModel.page, func.count()).where(
        EventModel.time >= start,
        EventModel.time <= finish,
        EventModel.page.in_(pages) 
        ).group_by(bucket, EventModel.page).order_by(bucket.asc()).limit(10)
    compile_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compile_query)
    results = session.exec(query).all()
    pprint(results)

SELECT time_bucket('1 day'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time >= '2026-01-13 07:39:00.091892+00:00' AND eventmodel.time <= '2026-01-13 08:40:00.091899+00:00' AND eventmodel.page IN ('/about', '/contact', '/home', '/products', '/services') GROUP BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 day'::interval, eventmodel.time) ASC
[]
